In [1]:
confidence=0.95

In [7]:
'''
Confidence intervals can then be obtained by first pooling for each sample 
the read counts of the mutations that belong to the same cluster followed by 
using a beta distribution. Please see the supplement of the MACHINA paper for more details.
'''
import sys
import os
os.chdir('/Users/divyakoyyalagunta/Desktop/Cornell_Research/Morris_Lab/metastatic_history_reconstruction_git/')

MACHINA_DATA_DIR = '/Users/divyakoyyalagunta/Desktop/Cornell_Research/Morris_Lab/machina/data/'
SIM_DATA_DIR = os.path.join(MACHINA_DATA_DIR, "sims")
SEED = 907
reads_filename = os.path.join(SIM_DATA_DIR, f"m5/M/reads_seed{SEED}.tsv")
cluster_filename = os.path.join(SIM_DATA_DIR,f"m5/M/clustering_observed_seed{SEED}.txt")

import pandas as pd 
data = pd.read_table(reads_filename, skiprows=3)
data

,#sample_index,sample_label,anatomical_site_index,anatomical_site_label,character_index,character_label,ref,var
0,0,P_0,0,P,0,0,97,85
1,0,P_0,0,P,1,1,104,84
2,0,P_0,0,P,2,2,202,0
3,0,P_0,0,P,3,3,167,33
4,0,P_0,0,P,4,4,212,0
...,...,...,...,...,...,...,...,...
506,6,M5_0,5,M5,68,76,112,81
507,6,M5_0,5,M5,69,77,130,93
508,6,M5_0,5,M5,70,80,103,67
509,6,M5_0,5,M5,71,81,183,0


In [8]:
# To obtain mutation clusters and their frequencies, we use the clustering procedure of AncesTree

# To infer a confidence interval on the frequency of mutation cluster C in each sample p,
# we combine the read counts for all mutations in the same mutation cluster C, 
# yielding a combined variant read count and combined total read count (see MACHINA supplement A.1.2 for more info)


# Map variant IDs to observed clusters IDs
variant_idx_to_cluster_idx_map = {}
with open(cluster_filename) as f:
    for i,line in enumerate(f):
        line = map(int, line.strip().split(';'))
        for l in line: variant_idx_to_cluster_idx_map[l] = i

variants = data['character_index'].unique()
sample_labels = data['sample_label'].unique()
print(data.head())

data['cluster'] = data['character_label'].apply(lambda x: variant_idx_to_cluster_idx_map[x])
data

# print("num variants:", len(variants))
# print("sample labels:", sample_labels)

# # Create a dataframe with the reference and variant reads for each variant ID
# cols = ["cluster"] + ['ref-'+c for c in sample_labels] + ['var-'+c for c in sample_labels]
# print(cols)

# pooled_data = []
# indices = []
# for v in variants:
#     if int(v) in variant_idx_to_cluster_idx_map:
#         cluster_idx = variant_idx_to_cluster_idx_map[int(v)]
#         row = [cluster_idx]
#         variant_subset = data[data['character_index']==v]
#         for col in ["ref", "var"]:
#             for sample_label in sample_labels:
#                 var_sample_sub = variant_subset[variant_subset['sample_label'] == sample_label]
#                 row.append(int(var_sample_sub[col]))
#         pooled_data.append(row)
#         indices.append(int(v))

# pooled_df = pd.DataFrame(pooled_data, index=indices, columns=cols)
# pooled_df   

   #sample_index sample_label  anatomical_site_index anatomical_site_label  \
0              0          P_0                      0                     P   
1              0          P_0                      0                     P   
2              0          P_0                      0                     P   
3              0          P_0                      0                     P   
4              0          P_0                      0                     P   

   character_index  character_label  ref  var  
0                0                0   97   85  
1                1                1  104   84  
2                2                2  202    0  
3                3                3  167   33  
4                4                4  212    0  


,#sample_index,sample_label,anatomical_site_index,anatomical_site_label,character_index,character_label,ref,var,cluster
0,0,P_0,0,P,0,0,97,85,0
1,0,P_0,0,P,1,1,104,84,0
2,0,P_0,0,P,2,2,202,0,1
3,0,P_0,0,P,3,3,167,33,3
4,0,P_0,0,P,4,4,212,0,2
...,...,...,...,...,...,...,...,...,...
506,6,M5_0,5,M5,68,76,112,81,13
507,6,M5_0,5,M5,69,77,130,93,13
508,6,M5_0,5,M5,70,80,103,67,13
509,6,M5_0,5,M5,71,81,183,0,14


In [38]:
ctable = pooled_df.groupby('cluster').sum()

global corrected_confidence
nsamples = len([c for c in ctable.columns if c.startswith('ref')])
nclusters = len(ctable)
corrected_confidence = 1-((1.-confidence)/(nsamples*nclusters))
print(corrected_confidence)

assert(corrected_confidence > confidence)
assert(corrected_confidence < 1.0)

0.9997916666666666


In [45]:
import numpy
from scipy.stats import beta
from scipy.stats import norm

def binomial_hpdr(n, N, pct, a=1, b=1, n_pbins=1e3):
    """
    Function computes the posterior mode along with the upper and lower bounds of the
    **Highest Posterior Density Region**.

    Parameters
    ----------
    n: number of successes 
    N: sample size 
    pct: the size of the confidence interval (between 0 and 1)
    a: the alpha hyper-parameter for the Beta distribution used as a prior (Default=1)
    b: the beta hyper-parameter for the Beta distribution used as a prior (Default=1)
    n_pbins: the number of bins to segment the p_range into (Default=1e3)

    Returns
    -------
    A tuple that contains the mode as well as the lower and upper bounds of the interval
    (mode, lower, upper)

    """
    # fixed random variable object for posterior Beta distribution
    rv = beta(n+a, N-n+b)
    # determine the mode and standard deviation of the posterior
    stdev = rv.stats('v')**0.5
    mode = (n+a-1.)/(N+a+b-2.)
    # compute the number of sigma that corresponds to this confidence
    # this is used to set the rough range of possible success probabilities
    n_sigma = numpy.ceil(norm.ppf( (1+pct)/2. ))+1
    # set the min and max values for success probability 
    max_p = mode + n_sigma * stdev
    if max_p > 1:
        max_p = 1.
    min_p = mode - n_sigma * stdev
    if min_p > 1:
        min_p = 1.
    # make the range of success probabilities
    p_range = numpy.linspace(min_p, max_p, int(n_pbins+1))
    # construct the probability mass function over the given range
    if mode > 0.5:
        sf = rv.sf(p_range)
        pmf = sf[:-1] - sf[1:]
    else:
        cdf = rv.cdf(p_range)
        pmf = cdf[1:] - cdf[:-1]
    # find the upper and lower bounds of the interval 
    sorted_idxs = numpy.argsort( pmf )[::-1]
    cumsum = numpy.cumsum( numpy.sort(pmf)[::-1] )
    j = numpy.argmin( numpy.abs(cumsum - pct) )
    upper = p_range[ (sorted_idxs[:j+1]).max()+1 ]
    lower = p_range[ (sorted_idxs[:j+1]).min() ]    

    return (mode, lower, upper)

In [49]:
def get_ub(row, sam):
    v=binomial_hpdr(row['var-'+sam], row['var-'+sam]+row['ref-'+sam], corrected_confidence)
    return v[2]
    

def get_lb(row, sam):
    v=binomial_hpdr(row['var-'+sam], row['var-'+sam]+row['ref-'+sam], corrected_confidence)
    mval = v[1]
    #if mval < 0.01: mval = 0
    return mval

def get_mean(row, sam):
    v=binomial_hpdr(row['var-'+sam], row['var-'+sam]+row['ref-'+sam], corrected_confidence)
    mval = v[0]
    return mval

ctable = pooled_df.groupby('cluster').sum()
for sample in sample_labels:
    ctable['ub-'+sample]= ctable.apply(get_ub, args=[sample], axis=1)
    ctable['lb-'+sample]= ctable.apply(get_lb, args=[sample], axis=1)
    ctable[sample]= ctable.apply(get_mean, args=[sample], axis=1)
ctable

,ref-P_0,ref-P_1,ref-M1_0,ref-M2_0,ref-M3_0,ref-M4_0,ref-M5_0,ref-M6_0,ref-M7_0,ref-M8_0,...,M5_0,ub-M6_0,lb-M6_0,M6_0,ub-M7_0,lb-M7_0,M7_0,ub-M8_0,lb-M8_0,M8_0
cluster,,,,,,,,,,,,,,,,,,,,,
0,90,80,102,91,113,98,98,86,127,101,...,0.502538,0.681873,0.421965,0.554404,0.562973,0.322995,0.440529,0.610723,0.350176,0.479381
1,297,268,323,277,291,302,327,279,307,332,...,0.226950,0.364333,0.196988,0.275325,0.326088,0.169545,0.241975,0.313606,0.164066,0.233256
2,507,477,544,552,398,508,500,410,396,503,...,0.192246,0.415392,0.275247,0.342949,0.408185,0.265741,0.334454,0.216122,0.106368,0.156040
3,190,209,204,201,231,204,196,101,96,203,...,0.000000,0.589147,0.323682,0.454054,0.676449,0.430297,0.555556,0.024272,0.000000,0.000000
4,109,91,203,191,207,96,100,206,187,124,...,0.481865,0.023923,0.000000,0.000000,0.026315,0.000000,0.000000,0.578056,0.337830,0.456140
5,175,194,199,95,181,183,199,167,214,182,...,0.000000,0.029411,0.000000,0.000000,0.023041,0.000000,0.000000,0.027027,0.000000,0.000000
6,367,306,405,316,378,466,395,390,402,407,...,0.000000,0.012723,0.000000,0.000000,0.012346,0.000000,0.000000,0.012195,0.000000,0.000000
7,203,200,212,159,210,184,200,197,199,175,...,0.000000,0.025000,0.000000,0.000000,0.024752,0.000000,0.000000,0.028089,0.000000,0.000000
8,199,203,173,203,195,196,232,185,192,199,...,0.000000,0.026595,0.000000,0.000000,0.025641,0.000000,0.000000,0.024752,0.000000,0.000000


In [52]:
def get_ub(row, sam):
    v=binomial_hpdr(row['var-'+sam], row['var-'+sam]+row['ref-'+sam], corrected_confidence)
    return v[2]
    

def get_lb(row, sam):
    v=binomial_hpdr(row['var-'+sam], row['var-'+sam]+row['ref-'+sam], corrected_confidence)
    mval = v[1]
    if mval < 0.01: mval = 0
    return mval

def get_mean(row, sam):
    v=binomial_hpdr(row['var-'+sam], row['var-'+sam]+row['ref-'+sam], corrected_confidence)
    mval = v[0]
    return mval

ctable_cutoff = pooled_df.groupby('cluster').sum()
for sample in sample_labels:
    ctable_cutoff['ub-'+sample]= ctable.apply(get_ub, args=[sample], axis=1)
    ctable_cutoff['lb-'+sample]= ctable.apply(get_lb, args=[sample], axis=1)
    ctable_cutoff[sample]= ctable.apply(get_mean, args=[sample], axis=1)
ctable_cutoff

,ref-P_0,ref-P_1,ref-M1_0,ref-M2_0,ref-M3_0,ref-M4_0,ref-M5_0,ref-M6_0,ref-M7_0,ref-M8_0,...,M5_0,ub-M6_0,lb-M6_0,M6_0,ub-M7_0,lb-M7_0,M7_0,ub-M8_0,lb-M8_0,M8_0
cluster,,,,,,,,,,,,,,,,,,,,,
0,90,80,102,91,113,98,98,86,127,101,...,0.502538,0.681873,0.421965,0.554404,0.562973,0.322995,0.440529,0.610723,0.350176,0.479381
1,297,268,323,277,291,302,327,279,307,332,...,0.226950,0.364333,0.196988,0.275325,0.326088,0.169545,0.241975,0.313606,0.164066,0.233256
2,507,477,544,552,398,508,500,410,396,503,...,0.192246,0.415392,0.275247,0.342949,0.408185,0.265741,0.334454,0.216122,0.106368,0.156040
3,190,209,204,201,231,204,196,101,96,203,...,0.000000,0.589147,0.323682,0.454054,0.676449,0.430297,0.555556,0.024272,0.000000,0.000000
4,109,91,203,191,207,96,100,206,187,124,...,0.481865,0.023923,0.000000,0.000000,0.026315,0.000000,0.000000,0.578056,0.337830,0.456140
5,175,194,199,95,181,183,199,167,214,182,...,0.000000,0.029411,0.000000,0.000000,0.023041,0.000000,0.000000,0.027027,0.000000,0.000000
6,367,306,405,316,378,466,395,390,402,407,...,0.000000,0.012723,0.000000,0.000000,0.012346,0.000000,0.000000,0.012195,0.000000,0.000000
7,203,200,212,159,210,184,200,197,199,175,...,0.000000,0.025000,0.000000,0.000000,0.024752,0.000000,0.000000,0.028089,0.000000,0.000000
8,199,203,173,203,195,196,232,185,192,199,...,0.000000,0.026595,0.000000,0.000000,0.025641,0.000000,0.000000,0.024752,0.000000,0.000000


In [55]:
def get_vaf(row, sam):
    return float(row['var-'+sam])/float(row['var-'+sam]+row['ref-'+sam])

#ctable_cutoff = table.groupby('cluster').mean()
vafs = pd.DataFrame()
for sample in sample_labels:
    vafs[sample] = pooled_df.apply(get_vaf, args=[sample], axis=1)
vafs['cluster'] = pooled_df['cluster']

In [59]:
rows = ["5 #anatomical sites\n5 #samples\n9 #mutations\n#sample_index\tsample_label\tanatomical_site_index\tanatomical_site_label\tcharacter_index\tcharacter_label\tf_lb\tf_ub\n",]

def print_char(row, sam):
    return "\t".join(map(str,[i, sam, i, sam, row.name, str(row.name), max(row['lb-'+sam] * 2, 0), min(1, 2 * row['ub-'+sam])]))+"\n"

for i, sam in enumerate(sample_labels):
    rows += list(ctable_cutoff.apply(print_char, args=[sam], axis=1))

with open("test_"+str(confidence)+".tsv", 'w') as f:
    for line in rows:
        f.write(line)